In [1]:
from gevent import monkey
monkey.patch_all(thread=False)

from gevent.pool import Pool
pool = Pool(5)

from gevent.lock import BoundedSemaphore
semaphore_correct, semaphore_error, semaphore_remain = BoundedSemaphore(1), BoundedSemaphore(1), BoundedSemaphore(1)

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import os
import shutil

directory_out = './file/4.vehicle'

if os.path.exists(directory_out):
    shutil.rmtree(directory_out)
os.makedirs(directory_out)

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

from tqdm import tqdm
import re

# = = = = = = = = = = = = = = =

directory_in = './file/1.part'

list_file = [file for file in sorted(os.listdir(directory_in)) if file not in ['.DS_Store'] and os.path.isfile(f'{directory_in}/{file}')]
for file in list_file:

    # = = = = = = = = = = = = = = =

    list_dict_correct, list_series_error, df_input = [], [], pd.read_excel(f'{directory_in}/{file}',
                                                                           header=0,
                                                                           dtype=str).fillna('')

    count_remain = len(df_input)

    print(f'总数量：{count_remain}')
    print()

    # = = = = = = = = = = = = = = =

    def crawler(crawler_tuple):
        global list_dict_correct, list_series_error, count_remain

        crawler_status, (crawler_index, crawler_series) = False, crawler_tuple

        # = = = = = = = = = = = = = = =

        try:
            count_retry = 0
            while True:
                count_retry += 1

                try:
                    url_check = crawler_series['[UAC] Url']

                    resp = requests.get(f'https://uacparts.com/Catalog/Buyers?part={crawler_series['[UAC] Part Number']}',
                                        headers=get_header(),
                                        proxies=get_proxy(),
                                        timeout=(5, 15))

                    if resp.status_code == 200:
                        soup = BeautifulSoup(resp.text, 'lxml')
                        html = etree.HTML(str(soup))

                        if html.xpath('//form[@action="/Catalog/Buyers"]'):
                            break
                except KeyboardInterrupt:
                    break
                except:
                    continue

            # = = = = = = = = = = = = = = =

            list_tr = html.xpath('//tr[@class="awe-row"]')

            # = = = = = = = = = = = = = = =

            list_dict_correct_temp = []
            for i, tr in enumerate(list_tr):
                list_td = tr.xpath('./td')
                list_dict_correct_temp.append({'[UAC] Part Number': crawler_series['[UAC] Part Number'],
                                               'Vehicle No': i + 1,
                                               'Year': int(list_td[0].xpath('./text()')[0]) if list_td[0].xpath('./text()') else '',
                                               'Make': list_td[1].xpath('./text()')[0] if list_td[1].xpath('./text()') else '',
                                               'Model': list_td[2].xpath('./text()')[0] if list_td[2].xpath('./text()') else '',
                                               'Submodel': list_td[3].xpath('./text()')[0] if list_td[3].xpath('./text()') else '',
                                               'Engine': list_td[4].xpath('./text()')[0] if list_td[4].xpath('./text()') else ''})

            # = = = = = = = = = = = = = = =

            with semaphore_correct:
                list_dict_correct.extend(list_dict_correct_temp)

            # = = = = = = = = = = = = = = =

            crawler_status = True
        except:
            crawler_series['Check_Url'] = url_check

            with semaphore_error:
                list_series_error.append(crawler_series)

        # = = = = = = = = = = = = = = =

        if list_file.index(file) % 3 == 0 and list_file.index(file) != 0 and count_remain == 1 or count_remain % 300 == 0 and count_remain != 0:
            clear_output()
        print(f'''[{crawler_status}] - [请求{count_retry}次] - [剩余{count_remain - 1}条] - [{file.removesuffix('.xlsx')}（{(list_file.index(file) + 1) / len(list_file) * 100:.2f}%）] - [{datetime.now().strftime('%Y/%m/%d %H:%M:%S')}] - {crawler_series['[UAC] Part Number']}. {url_check}\n''')

        with semaphore_remain:
            count_remain -= 1

    # = = = = = = = = = = = = = = =

    pool.map(crawler, df_input.iterrows())

    print('输出ing...')
    print()
    if list_dict_correct:
        df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['[UAC] Part Number', 'Vehicle No'],
                                                                 ascending=[True, True],
                                                                 ignore_index=True)
        df_correct.to_excel(f'''{directory_out}/{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
    if list_series_error:
        df_error = pd.DataFrame(list_series_error).sort_values(by=['[UAC] Part Number'],
                                                               ascending=[True],
                                                               ignore_index=True)
        df_error.to_excel(f'''./file/{file.removesuffix('.xlsx')}-vehicle_error.xlsx''', index=False)
        print('爬虫存在error')
        print()

# = = = = = = = = = = = = = = =

list_file = [file for file in sorted(os.listdir(directory_out)) if file not in ['.DS_Store'] and os.path.isfile(f'{directory_out}/{file}')]
for file in tqdm(list_file, desc='Progress', ncols=77):
    os.rename(f'{directory_out}/{file}',
              f'''{directory_out}/{re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '-vehicle.xlsx', file)}''')

print('Done ~')

[True] - [请求3次] - [剩余299条] - [fuck_new（100.00%）] - [2024/10/28 14:47:36] - RD 11212C. https://uacparts.com/Catalog/SearchIndex?search=RD 11212C

[True] - [请求10次] - [剩余298条] - [fuck_new（100.00%）] - [2024/10/28 14:47:36] - RD 11174KTC. https://uacparts.com/Catalog/SearchIndex?search=RD 11174KTC

[True] - [请求1次] - [剩余297条] - [fuck_new（100.00%）] - [2024/10/28 14:47:36] - RD 11211C. https://uacparts.com/Catalog/SearchIndex?search=RD 11211C

[True] - [请求23次] - [剩余296条] - [fuck_new（100.00%）] - [2024/10/28 14:47:36] - RD 11167C. https://uacparts.com/Catalog/SearchIndex?search=RD 11167C

[True] - [请求6次] - [剩余295条] - [fuck_new（100.00%）] - [2024/10/28 14:47:36] - RD 11207C. https://uacparts.com/Catalog/SearchIndex?search=RD 11207C

[True] - [请求71次] - [剩余294条] - [fuck_new（100.00%）] - [2024/10/28 14:47:36] - RD 10925C. https://uacparts.com/Catalog/SearchIndex?search=RD 10925C

[True] - [请求8次] - [剩余293条] - [fuck_new（100.00%）] - [2024/10/28 14:47:36] - RD 11208C. https://uacparts.com/Catalog/SearchIn

Progress: 100%|██████████████████████████████| 1/1 [00:00<00:00, 2941.31it/s]

Done ~
